In [2]:
import pandas as pd
import numpy as np
import json
from helper import fen_to_vector

In [9]:
def extract_cp(eval):
    first_pv = eval[0]["pvs"]
    try:
        return first_pv[0]["cp"]
    except KeyError:
        return first_pv[0]["mate"] #TODO: Make real big number pos or neg

    
def extract_depth(eval):
    depth = eval[0]["depth"]
    return depth

In [33]:
def json_to_dataframe(path, num):
  reader = pd.read_json(path, lines=True, chunksize=1000)
  df = pd.DataFrame(columns=["positions", "cp", "depth"])

  i = 0
  for line in reader: #Iterating by columns???
      #Turn FEN into bitboard array and add to dataframe
      bitboard = line["fen"].apply(fen_to_vector)
      df["positions"]= np.array(bitboard)

      #Extract the eval and depth, and add them to the dataframe
      #Only take the first eval and pv
      #Keep the highest cp value only, discard all other lines
      evals = line["evals"].apply(extract_cp)
      df["cp"] = np.array(evals)

      #Extract the depth and add it to the dataframe
      depth = line["evals"].apply(extract_depth)
      df["depth"] = np.array(depth)

      i += 1
      if i == num:
          break
  return df

json_to_dataframe("lichess_db_eval.jsonl", 1000)





,positions,cp,depth
0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, ...",98,24
1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, ...",31,27
2,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, ...",96,24
3,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, ...",80,24
4,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, ...",51,25
...,...,...,...
995,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, ...",-6,26
996,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, ...",0,26
997,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5,99
998,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, ...",97,23


In [35]:
df.to_csv("cleaned_data_1000.csv", encoding='utf-8', index=False)